In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from youtube_comment_downloader import YoutubeCommentDownloader
import pandas as pd

In [2]:
def scrape_youtube_comments(video_id, max_comments=500):
    downloader = YoutubeCommentDownloader()
    generator= downloader.get_comments_from_url(f"https://www.youtube.com/watch?v={video_id}")

    comments = []
    for count, comment in enumerate(generator):
        comments.append(comment['text'])
        if count >= max_comments - 1:
            break
    return comments

youtube_comments = scrape_youtube_comments("9ndC5FSLJBg", max_comments=500)
df_youtube = pd.DataFrame(youtube_comments, columns=['comment'])
df_youtube.head(500)

,comment
0,"they clearly felt the cold water, during beta ..."
1,"After playing the story, I still think the glo..."
2,"It is broken, as the reason most showcases ach..."
3,Being self-aware that you sound like a hoyo sh...
4,I don't really mind the passives even if they ...
...,...
474,@Ghosty2098 there was absolutely no leak that...
475,@Ghosty2098 Source: Trust me bro
476,@princekili1027 I know they be pissing me off
477,​ @Ghosty2098 where are your sources 💀\n\nI'v...


In [3]:
# Scrape Castorice's Fandom page for kit text and discussion sections (if available)

def scrape_fandom_wiki(url):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    content = []
    for p in soup.select('div.mw-parser-output > p'):
        content.append(p.get_text(strip=True))

    return "\n".join(content)

fandom_text = scrape_fandom_wiki("https://honkai-star-rail.fandom.com/wiki/Castorice")
print(fandom_text[:1000])

CastoriceServant of DeathSplash ArtIn-GameRarityCombat PathRemembranceCombat TypeQuantumBioFamilyVoiceTypePlayable CharacterStatusIn the nether realm permanently as ofTrailblaze MissionSea of Flowers, Adored Only by DeathOther TitlesMaiden of WarDaughter of the River of SoulsFactionsAmphoreus(on profile)Chrysos Heirs(demigod)[1]Styxia(hometown)[2]Aidonia(formerly)[3]Okhema[3]WorldAmphoreusMessagesCastorice's MessagesHow To ObtainBlossom to the BeyondRelease DateApril 09, 2025April 09, 20251 month agoRelativesPolyxia(younger twin sister) †[4][5]Queen ofStyxia(adoptive mother) †[6]Amunet(adoptive mother) †[6]EnglishMelody Muze[7]ChineseRuan Congqing (阮从青)[7]JapaneseSaito Chiwa (斎藤千和)[7]KoreanLee Serena (이세레나)[7]
Castoriceis a playablecharacterinHonkai: Star Rail.
The land that reveres death,Aidonia, where snow falls endlessly, has today drifted into a sweet slumber. Castorice, daughter of theRiver of Souls,Chrysos Heirin search of "Death"Coreflame, sets forth. You must guard the lament o

In [9]:
from selenium import webdriver
import time
import json

driver = webdriver.Chrome()
driver.get("https://www.hoyolab.com")
input("🟡 Please log in manually, then press ENTER to save cookies...")

with open("hoyolab_cookies.json", "w") as f:
    json.dump(driver.get_cookies(), f)

print("✅ Cookies saved.")
driver.quit()

✅ Cookies saved.


In [11]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Load cookies
with open("hoyolab_cookies.json", "r") as f:
    cookies = json.load(f)

# Setup browser
options = Options()
# options.add_argument("--headless")  # Optional
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# Step 1: Open base domain to set cookies
driver.get("https://www.hoyolab.com")
time.sleep(3)

# Step 2: Add cookies
for cookie in cookies:
    try:
        driver.add_cookie({
            "name": cookie["name"],
            "value": cookie["value"],
            "domain": cookie.get("domain", ".hoyolab.com"),
            "path": cookie.get("path", "/")
        })
    except Exception as e:
        print(f"⚠️ Skipping cookie {cookie['name']}: {e}")

# Step 3: Go to the article
driver.get("https://www.hoyolab.com/article/37996979")
time.sleep(5)

# Scroll to load comments
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_attempts = 0
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        scroll_attempts += 1
        if scroll_attempts >= 2:
            break
    else:
        scroll_attempts = 0
        last_height = new_height

# Extract visible comments
comment_elements = driver.find_elements(By.CSS_SELECTOR, ".mhy-reply-list > div")
comments = []
for el in comment_elements:
    try:
        user = el.find_element(By.CSS_SELECTOR, ".user-name").text.strip()
        comment = el.find_element(By.CSS_SELECTOR, ".reply-content__content").text.strip()
        comments.append({"user": user, "comment": comment})
    except Exception as e:
        continue

# Save results
with open("hoyolab_comments.json", "w", encoding="utf-8") as f:
    json.dump(comments, f, ensure_ascii=False, indent=2)

print(f"✅ Scraped {len(comments)} comments. Saved to hoyolab_comments.json")
driver.quit()


✅ Scraped 0 comments. Saved to hoyolab_comments.json
